# Еще один способ задать алгебраический тип

In [1]:
data Human = Student
              String -- name
              Int -- course
              String -- faculty
              Int -- age
              | Lecturer
              String -- name
              String -- faculty
              Int -- age deriving Show

In [4]:
h = Student "Ilya" 3 "arts" 70

getName :: Human -> String
getName (Student name _ _ _) = name
getName (Lecturer name _ _) = name

getName h

"Ilya"

Другой способ, можно именовать поля:

In [14]:
data Human = Student {name :: String, course :: Int, faculty :: String, age :: Int} |
             Lecturer {name :: String, faculty :: String, age :: Int} deriving Show
             
h = Student {name = "Ilya", course = 3, faculty = "arts", age = 70}
--s = Lecturer {name = "Pavel", faculty = "arts", age = 170}
s = Lecturer "Pavel" "arst" 170 -- можно по-старому
 

h
s

Student {name = "Ilya", course = 3, faculty = "arts", age = 70}

Lecturer {name = "Pavel", faculty = "arst", age = 170}

Мы получаем сразу функции для определения значений полей, и получаем возможность менять значения полей. Значения полей работают как функции:

In [21]:
-- получить значение полей

-- name :: Human -> String
name h
age s

-- заменить значения:
h2 = h {course = 4}

h -- старое значение не меняется
h2 -- создаётся новое значение

-- можно изменить сразу несколько полей
s {age = 180, faculty = "arts and arts"}

"Ilya"

170

Student {name = "Ilya", course = 3, faculty = "arts", age = 70}

Student {name = "Ilya", course = 4, faculty = "arts", age = 70}

Lecturer {name = "Pavel", faculty = "arts and arts", age = 180}

# Функторы, Аппликативные функторы, Монады

In [27]:
add1 = (+1)
add1 $ 42

-- функтор список
add1 <$> [10, 20, 30]

-- функтор MayBe
add1 <$> Just 42
add1 <$> Nothing

-- функтор Either e a = Left e | Right a
add1 <$> Right 42
add1 <$> Left "возникла ошибка при вычислении ответа на вопрос о смысле ..."

Line 2: Redundant $
Found:
add1 $ 42
Why not:
add1 42

43

[11,21,31]

Just 43

Nothing

Right 43

Left "\1074\1086\1079\1085\1080\1082\1083\1072 \1086\1096\1080\1073\1082\1072 \1087\1088\1080 \1074\1099\1095\1080\1089\1083\1077\1085\1080\1080 \1086\1090\1074\1077\1090\1072 \1085\1072 \1074\1086\1087\1088\1086\1089 \1086 \1089\1084\1099\1089\1083\1077 ..."

Задача, написать функцию `maybeSum :: Maybe Int -> Maybe Int -> Maybe Int`

In [30]:
maybeSum :: Maybe Int -> Maybe Int -> Maybe Int
maybeSum Nothing _ = Nothing
maybeSum _ Nothing = Nothing
maybeSum (Just a) (Just b) = Just (a + b)

maybeSum (Just 2) (Just 3)

Just 5

In [32]:
(+) <$> Just 2 <*> Just 3

Just 5

Другой пример, давайте вычислим `map` для значений внутри `Maybe`: `Just (+1)` `Just [10, 20, 30]`

In [33]:
plus1 = Just (+1)
list1 = Just [10, 20, 30]

-- map <$> plus1 -- Just (map (+1))
map <$> plus1 <*> list1

Just [11,21,31]

Попробуем сделать это же, но со списками вместо Maybe.
Физический смысл Maybe — значение, которого может не быть из-за ошибки в вычислениях.
Физический смысл `[a]` — это недетерминированные вычисления. Т.е. результат выполнения функций может быть разным, при нескольких вызовах, это моделируется тем, что значение является списком возножных значений:

In [36]:
-- coin :: Int -- невозможно в Haskell сделать так, чтобы иногда было 0, иногда 1
-- настоящий генератор в Haskell: coin :: State -> (Int, State)
coin :: [Int]
coin = [0, 1]

randomFromRange :: Int -> Int -> [Int]
randomFromRange a b = [a..b]

randomFromRange 10 20

[10,11,12,13,14,15,16,17,18,19,20]

Хочу получить два случайных числа от 1 до 6 (бросить кости) и сложить их (узнать сумму значений на костях):

In [38]:
die1 = randomFromRange 1 6 -- [1 2 3 4 5 6]
die2 = randomFromRange 1 6 -- [1 2 3 4 5 6]

-- 100% аналогично сложению внутри Maybe
(+) <$> die1 <*> die2

[2,3,4,5,6,7,3,4,5,6,7,8,4,5,6,7,8,9,5,6,7,8,9,10,6,7,8,9,10,11,7,8,9,10,11,12]

Для списков `[f1, f2] <*> [a, b, c]` получается `[f1 a, f1 b, f1 c, f2 a, f2 b, f2 c]`:

In [42]:
[(+1), (*2)] <*> [10, 20, 30, 40]


[map (+1), map (*2)] <*> [[1, 2, 3], [10, 20, 30], [100, 200, 300]]

[11,21,31,41,20,40,60,80]

[[2,3,4],[11,21,31],[101,201,301],[2,4,6],[20,40,60],[200,400,600]]

Существует функция `liftA2 :: Applicative f => (a -> b -> c) -> f a -> f b -> f c`

In [46]:
import Control.Applicative -- надо подключить модуль, чтобы использовать liftA2

--эквивалентно
(+) <$> Just 2 <*> Just 3

liftA2 (+) (Just 2) (Just 3)

Just 5

Just 5

Т.е. если функция `f` работает для обычных значений, то `liftA2 f` работает для завернутых значений. `liftA2 f x y = f <$> x <*> y`.

## Аппликативный функтор

Это класс типов, для которых определены операции `<*>` и `pure`. Аппликативные функторы должны быть еще и обычными функторами, т.е. операция `fmap = <$>` для них тоже определена.

Функции
 * `fmap = <$>` применяет обычную функцию к завернутому значению
 * `<*>` применяет завернутую функцию к завернутому значению
 * `pure` превратить обычное значение в завернутое

In [52]:
-- Haskell должен понимать, какой получается тип результата, чтобы правильно обернуть
pure 42 :: [Int]
pure 42 :: Maybe Int

pure 42 ++ [10, 20, 30] -- тут понятно, что нужен [Int]

[42]

Just 42

[42,10,20,30]

Давайте свяжем pure, <$> и <*>

In [64]:
(+1) 42
(+1) $ 42
(+1) <$> Just 42
Just (+1) <*> Just 42 -- нужна завернутая функция
pure (+1) <*> Just 42

(+1) <$> [10, 20, 30]
[ (+1) ] <*> [10, 20, 30]
pure (+1) <*> [10, 20, 30] -- заверни функцию, примени к завернутому

-- теперь давайте позаворачиваем аргументы функции, а не функции
(+1) <$> (pure 42 :: [Int])
(+1) <$> (pure 42 :: Maybe Int)

Line 2: Redundant $
Found:
(+ 1) $ 42
Why not:
(+ 1) 42Line 5: Use <$>
Found:
pure (+ 1) <*> Just 42
Why not:
(+ 1) <$> Just 42Line 9: Use <$>
Found:
pure (+ 1) <*> [10, 20, 30]
Why not:
(+ 1) <$> [10, 20, 30]

43

43

Just 43

Just 43

...

[11,21,31]

[11,21,31]

[11,21,31]

[43]

Just 43

Законы аппликативных функторов. Любое определение аппликативного функтора должно вводить функции <*>, pure, <$>, для которых выполняется несколько законов:

```
pure f <*> a = f <$> a

```
далее по ссылке:

[https://hackage.haskell.org/package/base-4.16.0.0/docs/Control-Applicative.html#g:1](https://hackage.haskell.org/package/base-4.16.0.0/docs/Control-Applicative.html#g:1)